# Сбор данных с Avito: аренда квартир, комнат и домов

**Цель:** Собрать данные об объявлениях аренды квартир, комнат и домов с Avito с фильтром "Вынужденно покинувшим свои дома".

**Последовательность:**
1. Парсинг объявлений квартир
2. Парсинг объявлений комнат
3. Парсинг объявлений домов
4. Объединение всех данных в единый датафрейм
5. Сохранение результатов в CSV

**Параметры:**
- URL объявления
- Заголовок
- Цена
- Количество комнат
- Площадь
- Адрес
- "Вынужденно покинувшим свои дома"
- Время публикации
- Дата сбора данных

## 1. Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import time
import re
import random
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

from tqdm import tqdm

## 2. Настройка параметров парсинга

In [2]:
#урл для парсинга категорий
urls = {
    'apartments': 'https://www.avito.ru/all/kvartiry/sdam/na_dlitelnyy_srok-ASgBAgICAkSSA8gQ8AeQUg?context=H4sIAAAAAAAA_wEjANz_YToxOntzOjg6ImZyb21QYWdlIjtzOjc6ImNhdGFsb2ciO312FITcIwAAAA&f=ASgBAgICA0SSA8gQ8AeQUoyNFQI',
    'rooms': 'https://www.avito.ru/all/komnaty/sdam/na_dlitelnyy_srok-ASgBAgICAkSQA74QqAn2YA?cd=1&context=H4sIAAAAAAAA_wEjANz_YToxOntzOjg6ImZyb21QYWdlIjtzOjc6ImNhdGFsb2ciO312FITcIwAAAA&f=ASgBAgICA0SQA74QqAn2YIyNFQI',
    'houses': 'https://www.avito.ru/all/doma_dachi_kottedzhi/sdam/na_dlitelnyy_srok-ASgBAgICAkSUA9IQoAjIVQ?cd=1&context=H4sIAAAAAAAA_wEjANz_YToxOntzOjg6ImZyb21QYWdlIjtzOjc6ImNhdGFsb2ciO312FITcIwAAAA&f=ASgBAgICA0SUA9IQoAjIVYyNFQI'
}

max_pages = 3

## 3. Определение функций

### 3.1. Функция для извлечения данных из заголовка

In [3]:
def parse_title(title):
    result = {'rooms': None, 'area': None}
    
    if 'студия' in title.lower():
        result['rooms'] = 'студия'
    else:
        #формат: 2-к.
        m = re.search(r'(\d+)-к\.', title)
        if m:
            result['rooms'] = m.group(1)
    
    #формат: 50 м² или 50.5 м²
    m = re.search(r'(\d+[,.]?\d*)\s*м²', title)
    if m:
        result['area'] = m.group(1).replace(',', '.')
    
    return result

### 3.2. Функция для извлечения параметра "Вынужденно покинувшим свои дома"

In [4]:
def get_free_period_avito(driver):
    try:
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located(
            (By.XPATH, "//div[@data-marker='item-view/item-params']")))
        
        #Ищем все элементы <li> с параметрами
        params = driver.find_elements(
            By.XPATH, 
            "//li[contains(@class, 'params__paramsList__item')]")
        
        #Ключевые слова 
        keywords = ['вынужденно', 'покинувшим', 'временное']
        
        #Перебираем
        for param in params:
            text = param.get_attribute('textContent')
            
            if text and any(keyword in text.lower() for keyword in keywords):
                text = text.strip()
                
                if ':' in text:
                    return text.split(':', 1)[1].strip()
                return text
        
        return None
        
    except Exception as e:
        return None

### 3.3. Функция для извлечения адреса

In [5]:
def get_address_avito(driver):
    try:
        wait = WebDriverWait(driver, 10)
        address_elem = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//span[contains(@class, 'item-address__string')]")))
        return address_elem.get_attribute('textContent').strip()
    except Exception as e:
        return None

### 3.4. Функция для сбора ссылок на объявления с текущей страницы

In [6]:
def get_listing_links_with_time(driver):
    try:
        driver.execute_script("window.scrollTo(0, 1500)")
        time.sleep(1)
        
        wait = WebDriverWait(driver, 10)
        cards = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, 'div[data-marker="item"]')))
        
        links_data = []
        
        for card in cards:
            try:
                a = card.find_element(By.CSS_SELECTOR, 'a[href]')
                link = a.get_attribute('href')
                
                #Проверяем
                if link and any(cat in link for cat in ['komnaty', 'doma_dachi', 'kvartiry']):
                    clean_link = link.split('?')[0]
                    
                    time_ago = None
                    try:
                        time_elem = card.find_element(By.CSS_SELECTOR, 'div[class*="iva-item-dateInfoStep"] p')
                        time_ago = time_elem.text.strip()
                    except:
                        pass
                    
                    links_data.append({
                        'url': clean_link,
                        'time_ago': time_ago})
            except:
                continue
        
        #Убираем дубликаты
        unique_links = {}
        for item in links_data:
            if item['url'] not in unique_links:
                unique_links[item['url']] = item
        
        return list(unique_links.values())
    
    except Exception as e:
        print(f"Ошибка при сборе ссылок: {str(e)}")
        return []

### 3.5. Функция для парсинга деталей объявления

In [7]:
def parse_listing_details(driver, link_info, category):
    try:
        driver.get(link_info['url'])
        wait = WebDriverWait(driver, 15)
        
        #Заголовок
        title = wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, 'h1[data-marker="item-view/title-info"]'))).text
        
        title_params = parse_title(title)
        
        #Цена
        price = None
        try:
            price_elem = wait.until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'span[data-marker="item-view/item-price"]')))
            price = price_elem.get_attribute("content")
        except:
            try:
                price_elem = driver.find_element(By.CSS_SELECTOR, 'span[itemprop="price"]')
                price = price_elem.get_attribute("content")
            except:
                pass
        
        try:
            wait.until(EC.presence_of_element_located(
                (By.CSS_SELECTOR, '#bx_item-params > ul')))
            time.sleep(0.5)
        except:
            pass
        
        #Адрес
        address = get_address_avito(driver)
        
        #Параметр "Вынужденно покинувшим"
        free_period = get_free_period_avito(driver)
        
        return {
            "category": category,
            "url": link_info['url'],
            "title": title,
            "price": price,
            "rooms": title_params['rooms'],  #Берём из заголовка
            "area": title_params['area'],    #Берём из заголовка
            "free_period": free_period,
            "address": address,
            "time_ago": link_info['time_ago'],
            "collection_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    
    except Exception as e:
        print(f"Ошибка парсинга {link_info['url'][:50]}...: {str(e)[:80]}")
        return None

### 3.6. Функция для перехода на следующую страницу

In [8]:
def go_to_next_page(driver, current_page_url):
    try:
        #Возвращаемся на главную страницу
        driver.get(current_page_url)
        time.sleep(random.uniform(2, 3))
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        
        next_btn = None
        selectors = [
            (By.XPATH, "//li[contains(@class, 'listItem_arrow_next')]/a"),
            (By.CSS_SELECTOR, "li.styles-module-listItem_arrow_next-KKNyO > a"),
            (By.CSS_SELECTOR, "div.pagination-pagination-vjzAT nav ul li:last-child a")]
        
        for by_type, selector in selectors:
            try:
                elements = driver.find_elements(by_type, selector)
                if elements and elements[0].is_displayed():
                    next_btn = elements[0]
                    break
            except:
                continue
        
        if not next_btn:
            print("Кнопка не найдена")
            return False, None
        
        #Проверяем на блок
        btn_class = next_btn.get_attribute('class') or ''
        if 'disabled' in btn_class:
            print("Кнопка заблокирована")
            return False, None
        
        #Прокручиваем и кликаем
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_btn)
        time.sleep(1)
        
        #Получаем URL следующей страницы
        next_page_url = next_btn.get_attribute('href')
        
        next_btn.click()
        time.sleep(random.uniform(4, 6))
        
        return True, driver.current_url
    
    except Exception as e:
        print(f"Ошибка перехода на следующую страницу: {str(e)[:150]}")
        return False, None

## 4. Инициализация WebDriver

In [9]:
chrome_options = Options()
chrome_options.add_argument("start-maximized")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
chrome_options.page_load_strategy = 'eager'

try:
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )
    
    #Доп защита
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {
        "userAgent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'})
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
except Exception as e:
    print(f'Ошибка при инициализации WebDriver: {e}')

## 5. Основной цикл парсинга

In [10]:
#Словарь для хранения данных по категориям
all_categories_data = {
    'apartments': [],
    'rooms': [],
    'houses': []}

category_names = {
    'apartments': 'Квартиры',
    'rooms': 'Комнаты',
    'houses': 'Дома'}

print(f"Начало: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

#Парсинг каждой категории
for category_key, start_url in urls.items():
    category_name = category_names[category_key]
    
    print(f"Категория: {category_name.upper()}")
    
    driver.get(start_url)
    time.sleep(random.uniform(4, 6))
    
    current_listing_url = driver.current_url
    
    #Парсинг страниц
    for page_num in range(1, max_pages + 1):
        print(f"{category_name} - Страница {page_num}/{max_pages}")
        
        links_data = get_listing_links_with_time(driver)
        print(f"Найдено объявлений: {len(links_data)}")
        
        if not links_data:
            print("Ссылки не найдены")
            continue
        
        #Парсим каждое объявление
        print(f"Парсинг объявлений со страницы {page_num}")
        
        for i, link_info in enumerate(tqdm(links_data, desc=f"{category_name} - стр.{page_num}"), 1):
            data = parse_listing_details(driver, link_info, category_key)
            
            if data:
                all_categories_data[category_key].append(data)
            
            time.sleep(random.uniform(2, 4))
        
        print(f"Обработано объявлений со страницы {page_num}: {len(links_data)}")
        print(f"Всего собрано по категории '{category_name}': {len(all_categories_data[category_key])}")
        
        #Переход на следующую страницу
        if page_num < max_pages:
            print(f"Переход на страницу {page_num + 1}")
            success, next_url = go_to_next_page(driver, current_listing_url)
            
            if not success:
                print("Не удалось перейти на следующую страницу, завершаем парсинг категории")
                break
            
            #Обновляем URL для следующей итерации
            current_listing_url = next_url if next_url else driver.current_url
            print("Переход выполнен")

print(f"Окончание: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Cобрано объявлений:")
for cat_key, cat_name in category_names.items():
    print(f"  - {cat_name}: {len(all_categories_data[cat_key])}")
print(f"  - ВСЕГО: {sum(len(v) for v in all_categories_data.values())}")

Начало: 2025-12-14 15:06:15
Категория: КВАРТИРЫ
Квартиры - Страница 1/3
Найдено объявлений: 50
Парсинг объявлений со страницы 1


Квартиры - стр.1: 100%|██████████| 50/50 [04:39<00:00,  5.58s/it]


Обработано объявлений со страницы 1: 50
Всего собрано по категории 'Квартиры': 50
Переход на страницу 2
Переход выполнен
Квартиры - Страница 2/3
Найдено объявлений: 49
Парсинг объявлений со страницы 2


Квартиры - стр.2: 100%|██████████| 49/49 [04:16<00:00,  5.24s/it]


Обработано объявлений со страницы 2: 49
Всего собрано по категории 'Квартиры': 99
Переход на страницу 3
Кнопка не найдена
Не удалось перейти на следующую страницу, завершаем парсинг категории
Категория: КОМНАТЫ
Комнаты - Страница 1/3
Найдено объявлений: 50
Парсинг объявлений со страницы 1


...

Обработано объявлений со страницы 1: 50
Всего собрано по категории 'Дома': 50
Переход на страницу 2
Переход выполнен
Дома - Страница 2/3
Найдено объявлений: 50
Парсинг объявлений со страницы 2


Дома - стр.2: 100%|██████████| 50/50 [04:56<00:00,  5.92s/it]


Обработано объявлений со страницы 2: 50
Всего собрано по категории 'Дома': 100
Переход на страницу 3
Переход выполнен
Дома - Страница 3/3
Найдено объявлений: 43
Парсинг объявлений со страницы 3


Дома - стр.3: 100%|██████████| 43/43 [04:15<00:00,  5.95s/it]

Обработано объявлений со страницы 3: 43
Всего собрано по категории 'Дома': 143
Окончание: 2025-12-14 15:39:45
Cобрано объявлений:
  - Квартиры: 99
  - Комнаты: 96
  - Дома: 143
  - ВСЕГО: 338


## 6. Закрытие WebDriver

In [11]:
driver.quit()

## 7. Объединение данных в единый DataFrame

In [12]:
#Создаём отдельные DataFrame для каждой категории
df_apartments = pd.DataFrame(all_categories_data['apartments']) if all_categories_data['apartments'] else pd.DataFrame()
df_rooms = pd.DataFrame(all_categories_data['rooms']) if all_categories_data['rooms'] else pd.DataFrame()
df_houses = pd.DataFrame(all_categories_data['houses']) if all_categories_data['houses'] else pd.DataFrame()
#Объединяем все данные
dataframes_to_concat = []
if not df_apartments.empty:
    dataframes_to_concat.append(df_apartments)
if not df_rooms.empty:
    dataframes_to_concat.append(df_rooms)
if not df_houses.empty:
    dataframes_to_concat.append(df_houses)

if dataframes_to_concat:
    df_combined = pd.concat(dataframes_to_concat, ignore_index=True)
    column_order = ['category', 'url', 'title', 'price', 'rooms', 'area', 'address', 'free_period', 'time_ago', 'collection_date']
    df_combined = df_combined[column_order]
else:
    print("Нет данных для объединения")
    df_combined = pd.DataFrame()

## 8. Просмотр результатов

In [13]:
df_combined

,category,url,title,price,rooms,area,address,free_period,time_ago,collection_date
0,apartments,https://www.avito.ru/kurskaya_oblast_zheleznog...,"3-к. квартира, 64 м², 3/5 эт.",20000,3,64,"Курская обл., Железногорск, Курская ул., 13к1",Бесплатно более 2 месяцев,2 ноября 13:33,2025-12-14 15:06:31
1,apartments,https://www.avito.ru/kineshma/kvartiry/1-k._kv...,"1-к. квартира, 45 м², 3/5 эт.",35000,1,45,"Ивановская обл., Кинешма, ул. Менделеева, 7",Бесплатно более 2 месяцев,8 сентября 21:30,2025-12-14 15:06:35
2,apartments,https://www.avito.ru/mahachkala/kvartiry/1-k._...,"1-к. квартира, 24 м², 1/1 эт.",12000,1,24,"Республика Дагестан, Махачкала, Горская ул.",Бесплатно на 1 неделю,23 сентября 2019,2025-12-14 15:06:41
3,apartments,https://www.avito.ru/kazan/kvartiry/2-k._kvart...,"2-к. квартира, 70 м², 7/8 эт.",41000,2,70,"Республика Татарстан (Татарстан), Казань, Минс...",Бесплатно более 2 месяцев,12 сентября 16:16,2025-12-14 15:06:47
4,apartments,https://www.avito.ru/orenburg/kvartiry/1-k._kv...,"1-к. квартира, 31 м², 4/9 эт.",17500,1,31,"Оренбургская обл., Оренбург, Алтайская ул., 2/2",Бесплатно на 2 недели,3 дня назад,2025-12-14 15:06:53
...,...,...,...,...,...,...,...,...,...,...
333,houses,https://www.avito.ru/tulskaya_oblast_yasnogors...,Коттедж 150 м² на участке 30 сот.,90000,None,150,"Тульская обл., Ясногорский р-н, муниципальное ...",Бесплатно на 1 неделю,14 октября 2020,2025-12-14 15:39:18
334,houses,https://www.avito.ru/irkutskaya_oblast_homutov...,Дом 143 м² на участке 7 сот.,40000,None,143,"Иркутская обл., Иркутский муниципальный округ,...",Бесплатно на 1 неделю,6 октября 19:39,2025-12-14 15:39:25
335,houses,https://www.avito.ru/rostov-na-donu/doma_dachi...,Коттедж 120 м² на участке 7 сот.,45000,None,120,"Ростовская обл., Мясниковский р-н, Чалтырское ...",Бесплатно более 2 месяцев,6 августа 22:15,2025-12-14 15:39:31
336,houses,https://www.avito.ru/kuybyshev/doma_dachi_kott...,Таунхаус 640 м² на участке 54 сот.,130000,None,640,"Новосибирская обл., Куйбышевский р-н, городско...",Бесплатно более 2 месяцев,3 октября 2016,2025-12-14 15:39:37


## 9. Сохранение результатов в CSV

In [14]:
output_filename = f'raw_data.csv'
df_combined.to_csv(output_filename, index=False, encoding='utf-8-sig')